In [1]:
from flask import Flask, jsonify, request, redirect
from flask_cors import CORS
import os
import vertexai
import json
from uuid import uuid4
from dotenv import load_dotenv
import os
import vertexai
import base64
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

In [7]:
PROJECT_ID="anagata-hackjakarta-429913"
LOCATION="us-central1"
MODEL_NAME="gemini-1.5-flash-001"

In [8]:
def generate(data):
    vertexai.init(project=PROJECT_ID, location=LOCATION)
    model = GenerativeModel(
        MODEL_NAME,
    )

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 0,
        "top_p": 0.95,
    }

    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text = """You are a real-time driver feedback system. Analyze the incoming sensor data to assess the driver\'s current behavior and provide immediate, actionable feedback.

    Feedback Types:
    -Visual
    -Auditory
    -Haptic

    Feedback Content:
    -Positive Reinforcement: Acknowledge safe driving practices (e.g., \"Smooth acceleration,\" \"Maintaining safe following distance\").
    -Constructive Criticism: Provide specific guidance for improvement when risky behavior is detected (e.g., \"Harsh braking detected, please ease off the brake,\" \"Sharp turn detected, please slow down before turning\").
    -Real-time Alerts: Immediately warn the driver of imminent dangers (e.g., \"Caution: Vehicle approaching quickly,\" \"Lane departure warning\").

    Additional Considerations:
    -Prioritization: Focus on the most critical feedback in real-time, avoiding information overload.
    -Frequency: Balance the frequency of feedback to avoid being annoying while ensuring timely alerts.
    -Data Privacy: Handle all driver data with strict confidentiality and adherence to privacy regulations.\"

    Severity Type:
    -Severe (e.g., rapid acceleration, hard braking)
    -Moderate (e.g., speeding, sharp turning)
    -Mild (e.g., frequent lane changes)
    -Info (e.g., smooth driving)

    Return the result JSON format, example:
    {
     \"trip_id\": \"12345\",
     \"user_id\": \"67890\",
    \"timestamp\": \"2024-07-24T15:30:00Z\",
     \"alerts\": [
      {
       \"type\": \"speeding\",
       \"description\": \"You were driving at 80 km/h in a 50 km/h zone.\",
       \"severity\": \"moderate\",
       \"suggestion\": \"Please slow down and adhere to the speed limit.\",
    \"feedback type\": [\"...\"]
      },
      {
       \"type\": \"rapid_acceleration\",
       \"description\": \"You rapidly accelerated from 0 to 60 km/h in 3 seconds.\",
       \"severity\": \"severe\",
       \"suggestion\": \"Avoid rapid acceleration to ensure smoother and safer driving.\",
    \"feedback type\": [\"...\"]
      },
      {
       \"type\": \"hard_braking\",
       \"description\": \"You applied hard braking from 70 km/h to 0 km/h within 2 seconds.\",
       \"severity\": \"moderate\",
       \"suggestion\": \"Anticipate stops earlier and apply brakes gradually.\",
    \"feedback type\": [\"...\"]
      },
      {
       \"type\": \"lane_change\",
       \"description\": \"Frequent lane changes detected over a short distance.\",
       \"severity\": \"mild\",
       \"suggestion\": \"Minimize lane changes to maintain a steady driving flow.\",
    \"feedback type\": [\"...\"]
      },
      {
       \"type\": \"sharp_turning\",
       \"description\": \"Sharp turning detected at high speed.\",
       \"severity\": \"moderate\",
       \"suggestion\": \"Slow down before turning to avoid sharp turns.\",
    \"feedback type\": [\"...\"]
      },
      {
       \"type\": \"smooth_driving\",
       \"description\": \"Maintain a smooth driving style for better safety.\",
       \"severity\": \"info\",
       \"suggestion\": \"Continue driving smoothly to improve fuel efficiency and safety.\",
    \"feedback type\": [\"...\"]
      }
     ]
    }

    Just return JSON format, don\'t give other reason. Please, be consistent!
    Now, it\'s your turn. Analyze this data!"""

    responses = model.generate_content(
        [text, data],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )
    
    return responses.candidates[0].content.parts[0].text

In [9]:
data = {
    "trip_id": "12345",
    "uid": "67890",
    "timestamp": "2024-07-27T12:34:56Z",
    "location": {
        "lat": 40.7128,
        "lon": -74.0060
    },
    "speed": {
        "current": 50.0
    },
    "accelerometer": {
        "x": 0.1,
        "y": 0.2,
        "z": 0.3
    },
    "gyroscope": {
        "roll": 1.0,
        "pitch": 0.5,
        "yaw": 0.3
    },
    "lane_change": {
        "count": 1
    },
    "turning": {
        "left_turns": 0,
        "right_turns": 1
    },
    "deceleration": {
        "count": 1,
        "average_deceleration": -0.5
    }
}

In [ ]:
def analyze():
    try:
        driving_data = request.get_json()

        try:
            feedback = generate(driving_data)
            return json.dumps(feedback, indent=4)
        except Exception as e:
            return json.dumps({"message": "Internal server error. Please try again later."}), 500
    
    except Exception as e:
        return json.dumps({"message": "Invalid data format."}), 400